# Clustering Crypto

In [1]:
# Initial imports
!pip install -U altair
import requests
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
from altair import Chart, X, Y, Axis, SortField
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

  Using cached altair-4.1.0-py3-none-any.whl (727 kB)
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


### Fetching Cryptocurrency Data

In [26]:
# Use the following endpoint to fetch json data
import json
from pprint import pprint

url = "https://min-api.cryptocompare.com/data/all/coinlist"
res = requests.get(url)
res
data = res.json()
json_data = json.dumps(data)

In [27]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.
df = pd.DataFrame(data["Data"])
df = df.T
url_request_df = df[['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','MaxSupply']]
url_request_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,True,PoW/PoS,0,0
300,300 token,N/A,True,N/A,300,300
365,365Coin,X11,True,PoW/PoS,0,0
404,404Coin,Scrypt,True,PoW/PoS,0,0
433,433 Token,N/A,False,N/A,NaN,NaN


In [28]:
df.keys()
# json file keys do not contain "TotalCoinSupply" column as in .csv file below. Further analysis will be using data from .csv file.

Index(['Id', 'Url', 'ImageUrl', 'ContentCreatedOn', 'Name', 'Symbol',
       'CoinName', 'FullName', 'Description', 'AssetTokenStatus', 'Algorithm',
       'ProofType', 'SortOrder', 'Sponsored', 'Taxonomy', 'Rating',
       'IsTrading', 'TotalCoinsMined', 'BlockNumber', 'NetHashesPerSecond',
       'BlockReward', 'BlockTime', 'AssetLaunchDate', 'MaxSupply',
       'MktCapPenalty', 'PlatformType', 'BuiltOn', 'SmartContractAddress',
       'DecimalPoints', 'Difficulty', 'IsUsedInDefi'],
      dtype='object')

In [29]:
# Alternatively, use the provided csv file:
# file_path = Path("Resources/crypto_data.csv")
crypto_df = pd.read_csv("Resources/crypto_data.csv")

# Create a DataFrame
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [30]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df = crypto_df.set_index(crypto_df.columns[0])
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,,
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [31]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df[crypto_df['IsTrading']==True]
print(crypto_df.shape)
crypto_df.head()

(1144, 6)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,,
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [32]:
# Keep only cryptocurrencies with a working algorithm
print(crypto_df['Algorithm'].isna())

Unnamed: 0
42      False
365     False
404     False
611     False
808     False
        ...  
SERO    False
UOS     False
BDX     False
ZEN     False
XBC     False
Name: Algorithm, Length: 1144, dtype: bool


In [33]:
# Remove the "IsTrading" column
crypto_df = crypto_df.drop(columns="IsTrading")
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [34]:
# Remove rows with at least 1 null value
crypto_df.isnull().sum()

CoinName             0
Algorithm            0
ProofType            0
TotalCoinsMined    459
TotalCoinSupply      0
dtype: int64

In [35]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df[crypto_df['TotalCoinsMined'] > 0]
crypto_df.shape

(532, 5)

In [36]:
# Drop rows where there are 'N/A' text values
crypto_df.dropna()
crypto_df.isna().sum()

CoinName           0
Algorithm          0
ProofType          0
TotalCoinsMined    0
TotalCoinSupply    0
dtype: int64

In [37]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coin_name_df = crypto_df["CoinName"]
coin_name_df.head()

Unnamed: 0
42        42 Coin
404       404Coin
1337    EliteCoin
BTC       Bitcoin
ETH      Ethereum
Name: CoinName, dtype: object

In [38]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df = crypto_df.drop(columns='CoinName')
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [39]:
# Create dummy variables for text features
crypto_df_dummies = pd.get_dummies(crypto_df, columns=['Algorithm', 'ProofType'])
crypto_df_dummies.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
# Standardize data
scaler = StandardScaler()
crypto_data_scaled = scaler.fit_transform(crypto_df_dummies)
print(crypto_data_scaled[0:5])

[[-0.11710817 -0.1528703  -0.0433963  -0.0433963  -0.0433963  -0.06142951
  -0.07530656 -0.0433963  -0.06142951 -0.06142951 -0.0433963  -0.0433963
  -0.19245009 -0.06142951 -0.09740465 -0.0433963  -0.11547005 -0.07530656
  -0.0433963  -0.0433963  -0.15191091 -0.0433963  -0.13118084 -0.0433963
  -0.0433963  -0.08703883 -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.06142951 -0.0433963  -0.08703883 -0.08703883 -0.08703883 -0.0433963
  -0.13118084 -0.13840913 -0.13840913 -0.0433963  -0.06142951 -0.0433963
  -0.07530656 -0.18168574 -0.0433963  -0.0433963  -0.0433963  -0.07530656
  -0.15826614 -0.31491833 -0.0433963  -0.08703883 -0.07530656 -0.06142951
   1.38675049 -0.0433963  -0.0433963  -0.06142951 -0.0433963  -0.0433963
  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.39879994 -0.0433963  -0.18168574 -0.0433963  -0.08703883 -0.08703883
  -0.10680283 -0.0433963  -0.13118084 -0.0433963  -0.0433963  -0.0433963
  -0.0433963  -0.07530656 -0.43911856 -0.04339

### Reducing Dimensions Using PCA

In [41]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(crypto_data_scaled)

In [42]:
# Create a DataFrame with the principal components data
pca_df = pd.DataFrame(crypto_pca, columns=["PCA 1", "PCA 2", "PCA 3"], index=crypto_df.index)
pca_df.head()

,PCA 1,PCA 2,PCA 3
Unnamed: 0,,,
42,-0.338641,1.044811,-0.435356
404,-0.321972,1.045053,-0.435631
1337,2.311816,1.695445,-0.533880
BTC,-0.147436,-1.323315,0.123726
ETH,-0.144915,-2.069679,0.281092


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [43]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(crypto_pca)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow
elbow_graph = alt.Chart(df_elbow).mark_line().encode(x='k', y='inertia').properties(title="Elbow Curve")

elbow_graph

alt.Chart(...)

Running K-Means with `k=<your best value for k here>`

In [44]:
# Initialize the K-Means model
model = KMeans(n_clusters=5, random_state=0)
# Fit the model
model.fit(pca_df)
# Predict clusters
predictions = model.predict(pca_df)
# Create a new DataFrame including predicted clusters and cryptocurrencies features
crypto_df['Class'] = model.labels_
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,Class
Unnamed: 0,,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,4
BTC,SHA-256,PoW,1.792718e+07,21000000,1
ETH,Ethash,PoW,1.076842e+08,0,1


In [45]:
crypto_df.head()
clustered_df = pd.concat([crypto_df, pca_df, coin_name_df], axis=1, join="inner")
clustered_df = clustered_df[['CoinName','Algorithm','ProofType','TotalCoinsMined','TotalCoinSupply','PCA 1', 'PCA 2', 'PCA 3', 'Class']]
clustered_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PCA 1,PCA 2,PCA 3,Class
Unnamed: 0,,,,,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42,-0.338641,1.044811,-0.435356,0
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.321972,1.045053,-0.435631,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359,2.311816,1.695445,-0.533880,4
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000,-0.147436,-1.323315,0.123726,1
ETH,Ethereum,Ethash,PoW,1.076842e+08,0,-0.144915,-2.069679,0.281092,1


### Visualizing Results

#### 2D-Scatter with Clusters

In [46]:
# Create a 2D-Scatter with the PCA data and the clusters
pca_graph = alt.Chart(
    clustered_df).mark_point().encode(
    x="PCA 1",
    y="PCA 2",
    tooltip=["CoinName", "Algorithm", "TotalCoinsMined", "TotalCoinSupply", "Class"],
    color= alt.Color("Class",scale=alt.Scale(scheme='dark2'))
).properties(title="PCA 1 vs PCA 2 ").interactive()
    
pca_graph

alt.Chart(...)

#### Table of Tradable Cryptocurrencies

In [47]:
# Table with tradable cryptos
display(clustered_df)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PCA 1,PCA 2,PCA 3,Class
Unnamed: 0,,,,,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42,-0.338641,1.044811,-0.435356,0
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.321972,1.045053,-0.435631,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359,2.311816,1.695445,-0.533880,4
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000,-0.147436,-1.323315,0.123726,1
ETH,Ethereum,Ethash,PoW,1.076842e+08,0,-0.144915,-2.069679,0.281092,1
...,...,...,...,...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000,2.463881,0.759841,-0.316889,4
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000,-0.336685,1.044703,-0.435368,0
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610,0.330889,-2.331691,0.286546,1


In [48]:
# Print the total number of tradable cryptocurrencies
total_tradable_cryptos = len(clustered_df)
print(f"Total number of tradable cryptos = {total_tradable_cryptos}")

Total number of tradable cryptos = 532


#### Scatter Plot with Tradable Cryptocurrencies

In [49]:
# Scale data to create the scatter plot
clustered_df["TotalCoinsMined"] = clustered_df["TotalCoinsMined"]/10000
clustered_df["TotalCoinSupply"] = clustered_df["TotalCoinSupply"].astype("float64")
clustered_df["TotalCoinSupply"] = clustered_df["TotalCoinSupply"]/10000

In [51]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
fig = alt.Chart(clustered_df).mark_point().encode(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    tooltip=["CoinName", "Algorithm", "TotalCoinsMined", "TotalCoinSupply", "Class"],
    color=alt.Color("Class",scale=alt.Scale(scheme='dark2'))
).properties(title="Total Coin Supply vs. Total Coins Mined (10,000s)").interactive()
    
fig

alt.Chart(...)